In [1]:
!apt-get install poppler-utils tesseract-ocr libtesseract-dev tesseract-ocr-rus

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev poppler-utils tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd tesseract-ocr-rus
0 upgraded, 8 newly installed, 0 to remove and 29 not upgraded.
Need to get 10.0 MB of archives.
After this operation, 36.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.3 [581 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 

In [7]:
# Install required libraries
!pip install -q transformers torch accelerate chromadb sentence-transformers langchain-community langchain-huggingface unstructured unstructured[pdf] unstructured[docx]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.2 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import torch
import os

In [4]:
model_name = "unsloth/gemma-2b-it"  # Replace with "gemma-7b" for the 7B version
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [51]:
RAG_CONFIG = {
    "data_dir": "/content/",          # Папка с документами
    "chunk_size": 1000,            # Размер фрагментов текста
    "chunk_overlap": 200,          # Перекрытие между фрагментами
    "k_results": 3,                # Количество возвращаемых фрагментов
    "embeddings_model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
}

In [52]:
# Инициализация RAG
def initialize_rag():
    # Загрузка документов
    loader = DirectoryLoader(RAG_CONFIG["data_dir"], glob="**/*.docx")
    documents = loader.load()

    # Разбиение на фрагменты
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=RAG_CONFIG["chunk_size"],
        chunk_overlap=RAG_CONFIG["chunk_overlap"]
    )
    texts = text_splitter.split_documents(documents)

    # Используем GPU для модели эмбеддингов
    embeddings = HuggingFaceEmbeddings(
        model_name=RAG_CONFIG["embeddings_model"],
        model_kwargs={"device": "cuda"}  # Указываем использование GPU
    )

    # Создание векторного хранилища
    vector_db = Chroma.from_documents(
        texts,
        embeddings,
        persist_directory="./rag_db"
    )
    return vector_db

# Проверяем и инициализируем RAG
if not os.path.exists("./rag_db"):
    print("Инициализация RAG системы...")
    vector_db = initialize_rag()
else:
    print("Загрузка существующей RAG базы...")
    embeddings = HuggingFaceEmbeddings(
        model_name=RAG_CONFIG["embeddings_model"],
        model_kwargs={"device": "cuda"}  # Указываем использование GPU
    )
    vector_db = Chroma(persist_directory="./rag_db", embedding_function=embeddings)

Загрузка существующей RAG базы...


<ipython-input-52-e23c55ff0e05>:38: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory="./rag_db", embedding_function=embeddings)


In [66]:
def add_documents_to_rag(vector_db, new_data_dir):
    # Загрузка новых документов
    loader = DirectoryLoader(new_data_dir, glob="**/*.docx")
    new_documents = loader.load()

    # Разбиение на фрагменты (используем те же параметры, что при инициализации)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=RAG_CONFIG["chunk_size"],
        chunk_overlap=RAG_CONFIG["chunk_overlap"]
    )
    new_texts = text_splitter.split_documents(new_documents)

    # Добавляем новые фрагменты в векторное хранилище
    vector_db.add_documents(new_texts)

    # Сохраняем изменения на диск
    vector_db.persist()
    print(f"Добавлено {len(new_texts)} новых фрагментов из директории {new_data_dir}")

# Пример использования после инициализации vector_db
add_documents_to_rag(vector_db, "./new_data")

Добавлено 16 новых фрагментов из директории ./new_data


<ipython-input-66-1b1d56835056>:17: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


In [67]:
# Функция для генерации ответа с RAG
def ask_question_with_rag(question, system_prompt=None):
    # Поиск релевантной информации
    docs = vector_db.similarity_search(question, k=RAG_CONFIG["k_results"])
    context = "\n\n".join([doc.page_content for doc in docs])

    # Формирование промпта
    prompt_parts = []
    if system_prompt:
        prompt_parts.append(f"System: {system_prompt}")
    if context:
        prompt_parts.append(f"Context: {context}")
    prompt_parts.append(f"User: {question}\nAssistant:")

    full_prompt = "\n\n".join(prompt_parts)

    # Генерация ответа
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=4096,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    # Декодирование ответа
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = full_response.split("Assistant:")[-1].strip()
    return answer

In [68]:
# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): GemmaRMSNorm((2048,), 

In [69]:
system_prompt = "Ты помощник по Уральскому Федеральному университету, который общается с пользователем. Общайся с ним только на русском языке. Объясняй максимально подробно, чтобы пользователь всё понял."

In [70]:
question = "Расскажи мне подробно про направление Программная инженерия"

In [71]:
answer = ask_question_with_rag(question, system_prompt)
print(f"Вопрос: {question}")
print(f"Ответ: {answer}")

Вопрос: Расскажи мне подробно про направление Программная инженерия
Ответ: Программная инженерия – это профессия, которая требует знания и навыков программирования. Она’s  упрощение задач, которые могут быть решены с помощью программирования. 

В этой профессии ты будет:

* Разработка алгоритмов и систем
* Разработка веб-приложений
* Разработка мобильных приложений
* Разработка и оптимизация кодов

Чтобы быть успешным программистом, нужно:

* Найти себе время, чтобы изучить программирование
* Изучить языки программирования C#, Java, C++ и Python
* Разработать навыки работы с базами данных
* Пройти собеседования

В дополнение к основным знаниям, программисты также должны иметь:

*  Профессиональные навыки, такие как управление проектами, менеджмент, маркетинг, бизнес-аналитика, и другие
*  Позитивный настрой и способность работать в команде

Программная инженерия – это очень interesante и перспективный профессия, которая требует от тебя усилий и dedication.


In [87]:
question = "Расскажи мне про институт"

In [88]:
answer = ask_question_with_rag(question, system_prompt)
print(f"Вопрос: {question}")
print(f"Ответ: {answer}")

Вопрос: Расскажи мне про институт
Ответ: ИРИТ-РТФ - это один из крупнейших институтов Урала, который специализируется на подготовке профессионалов ИТ-сферы, информационной безопасности и защиты информации. В институте можно найти множество возможностей для обучения и развития, включая онлайн-курсы, мастер-классы, конференции и исследования.
